In [6]:
%load_ext autoreload
%autoreload 2

%load_ext rpy2.ipython

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [2]:
import pandas as pd
import plotly.express as px

from src.report import conf_int, t_test, f_test, general, general_assessment, other
from src.stats import LinearRegression, add_constant

In [3]:
from data import DATA_FRAME

# Исходные данные

In [4]:
DATA_FRAME

,y,x_1,x_2,x_3
0,195.197059,168.756262,31.972479,46.746935
1,195.190125,168.663856,32.073664,46.010348
2,196.225667,170.136834,33.630488,46.477245
3,197.097511,170.662084,34.285954,46.660334
4,193.652921,167.539127,32.767683,47.977245
...,...,...,...,...
295,195.360906,169.267803,33.383489,45.590076
296,194.890106,168.369894,32.911007,47.821680
297,195.509979,170.538656,35.594192,48.438330
298,195.875307,170.125282,34.178079,46.502980


# Матрица парной корреляции

In [5]:
DATA_FRAME.corr()

,y,x_1,x_2,x_3
y,1.000000,0.804312,0.387970,-0.607512
x_1,0.804312,1.000000,0.669191,-0.442888
x_2,0.387970,0.669191,1.000000,-0.100470
x_3,-0.607512,-0.442888,-0.100470,1.000000


# Диаграммы рассеивания

In [11]:
px.scatter(DATA_FRAME, y='y', x='x_1', trendline='ols', trendline_color_override='orange')

In [12]:
px.scatter(DATA_FRAME, y='y', x='x_3', trendline='ols', trendline_color_override='orange')

# Парная линейная регрессия

## Python

In [16]:
Y = DATA_FRAME['y']
X = add_constant(DATA_FRAME['x_1'])

In [17]:
model = LinearRegression(Y, X)

In [18]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.647
Model:                            OLS   Adj. R-squared:                  0.646
Method:                 Least Squares   F-statistic:                     546.0
Date:                Sat, 16 Oct 2021   Prob (F-statistic):           2.47e-69
Time:                        09:37:50   Log-Likelihood:                -262.96
No. Observations:                 300   AIC:                             529.9
Df Residuals:                     298   BIC:                             537.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         60.8199      5.746     10.585      0.0

### Модель

In [19]:
general(model)

$ y_i = a + b_{1} x_{i1} + e_i $<br>
<br>
$ \hat{y_i} = \hat{a} + \hat{b_{1}} x_{i1} $<br>
<br>
$ \hat{y_i} = 60.820 + 0.794 x_{i1} $<br>
<br>
$ S_\hat{a} = 5.746 $<br>
$ S_{\hat{b_{1}}} = 0.034 $

### Общая оценка

In [20]:
general_assessment(model)

### Оценка среднеквадратического отклонения возмущений.<br>
<br>
$ \sqrt{\frac{1}{n - k - 1} \sum_{i=1}^{n}{(y_i - \hat{y_i}})^2} $<br>
<br>
$ S_{e} = 0.583 $<br>
$ 0 \% < 0.30_{S_{ei}} \% < 2 \% \rightarrow $ оценка: <b>10/10</b>.<br>
<br>
<br>
### Коэффициент детерминации.<br>
<br>
$ R^2 = 1 - \frac{\sum_{i=1}^{n}{(y_i - \hat{y_i})^2}}{\sum_{i=1}^{n}{(y_i - \bar{y})^2}}  = \frac{\sum_{i=1}^{n}{(\hat{y_i} - \bar{y})^2}}{\sum_{i=1}^{n}{(y_i - \bar{y})^2}}  = 1 - \frac{RSS}{TSS} = \frac{ESS}{TSS} $<br>
<br>
$ R^2 = 0.647 $<br>
$ 0.5 < 0.647 < 0.7 \rightarrow $ оценка: <b>3/10</b>.<br>
<br>
<br>
### Средняя относительная ошибка аппроксимации.<br>
<br>
$ A = \frac{1}{n} \sum_{i=1}^{n}{|\frac{y_i - \hat{y_i}}{y_i}|} \cdot 100 \% $<br>
<br>
$ A = 0.240 $<br>
$ 0 \% < 0.240 < 2 \% \rightarrow $ оценка: <b>10/10</b>.

### Оценка модели на значимость

In [21]:
f_test(model)

### F-критерий Фишера.
<br><br>
$ H_0: b_1 = ... = b_k = 0, \\ H_1: b_1^2 + ... + b_k^2 > 0. $
<br><br>
$$ F = \frac{\frac{R^2}{k}}{\frac{1 - R^2}{(n - k - 1)}} $$
<br><br>
$ F_{набл} = 545.994 $<br>
$ p\text{-}value_F = 0.000 $
<br><br>
$ \alpha = 0.05 $<br>
$ F_{табл_{0.05}} = 1.210 $
<br><br>
$ 2.47e-69 < 0.05 $<br>
$ p\text{-}value < \alpha \rightarrow $ гипотеза $ H_0 $ <b>отвергается</b> - модель в целом <b>значима</b>.

### Оценка значимости параметров модели

In [22]:
t_test(model)

### t-критерий Стьюдета для оценки значимости параметров модели.
<br><br>
$ H_0: b_i = 0, \\ H_1: b_i \neq 0. $
<br><br>
$$ t_{b_i} = \frac{\hat{b_i}}{S_{\hat{b_i}}} $$
<br><br>
$ \alpha = 0.05 $
<br>
$ t_{табл_{0.05}} = 1.968 $
<br>

<br>
$ t_{a} = 10.585 $, $ p\text{-}value_{t_{a}} = 0.000 $, $ p\text{-}value < \alpha \rightarrow $гипотеза $ H_0 $ <b>отвергается</b> - параметр <b>значим</b>.<br>$ t_{b_{1}} = 23.367 $, $ p\text{-}value_{t_{b_{1}}} = 0.000 $, $ p\text{-}value < \alpha \rightarrow $гипотеза $ H_0 $ <b>отвергается</b> - параметр <b>значим</b>.

### Доверительные интервалы

In [23]:
conf_int(model)

$ \hat{b_i} - \hat{\sigma} \cdot t_{табл} \leq b_i \leq \hat{b_i} + \hat{\sigma} \cdot t_{табл} $<br>
<br>
$ \alpha = 0.05 $<br>
<br>
$ a : (49.512; 72.128) $<br>
$ b_{1} : (0.727; 0.861) $

### Дополнительные оценки

In [24]:
other(model)

### Бета-коэффициенты.<br>
<br>
$ \beta_j = \hat{b_j} \cdot \frac{S_{x_{ij}}}{S_{y_i}} $<br>
<br>
$ \beta_{1} = 0.804 $<br>
<br>
### Дельта-коэффициенты.<br>
<br>
$ \Delta_j = r_{y_i x_{ij}} \cdot \frac{\hat{b_j}}{R^2} $<br>
<br>
$ \Delta_{1} = 1.000 $<br>
<br>
### Коэффициенты эластичности.<br>
<br>
$ Э_{j} = \hat{b_j} \cdot \frac{\bar{x_{ij}}}{\bar{y_i}} $<br>
<br>
$ Э_{1} = 0.688 $

### 

## R

In [ ]:
%Rpush DATA_FRAME

In [ ]:
%%R

library(lmtest)

m = lm(y ~ 1 + x_1, data = DATA_FRAME)

sm = summary(m)

sm

# Множественная линейная регрессия